<a href="https://colab.research.google.com/github/BelRom/PythonHW3/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Environment_example_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U kaggle_environments


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import random
import time

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

termcolor not installed, skipping dependency
No pygame installed, ignoring import


Опишем поведение агента, всегда играющего "камень" - это значение 0

In [3]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [4]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

1 Опишем поведение агента который всегда выбирает ножницы

In [5]:
%%writefile scissors_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def scissors_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


2 Опишем поведение агента который всегда выбирает бумагу

In [6]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


3 Опишем поведение агента который чередует ножницы и камень

In [7]:
%%writefile rock_scissors_agent.py

def rock_scissors_agent(observation, configuration):
    result = 0
    if observation.step % 2 == 0:
        result = 0
    else:
        result = 2
    return result



Overwriting rock_scissors_agent.py


4 Опишем поведение агента который чередует камень и бумагу

In [8]:
%%writefile rock_paper_agent.py

def rock_paper_agent(observation, configuration):
    result = 0
    if observation.step % 2 == 0:
        result = 0
    else:
        result = 1
    return result

Overwriting rock_paper_agent.py


5 Опишем поведение агента который последовательно выбирает камень, ножницы, бумагу

In [9]:
%%writefile order_rock_scissors_paper_agent.py

def order_rock_scissors_paper_agent(observation, configuration):
    result = 0
    if observation.step % 3 == 0:
        result = 0
    if observation.step % 2 == 0:
        result = 1
    else:
        result = 2
    return result

Overwriting order_rock_scissors_paper_agent.py


6 Опишем поведение агента который случайно выбирает любой элемент

In [10]:
%%writefile random_agent.py

import random

def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)

Overwriting random_agent.py


7 Опишем поведение агента который меняет действие на противоположное по отношению к опоненту

In [11]:
%%writefile oposite_agent.py

def oposite_agent(observation, configuration):
    if observation.step == 0:
        return 2
    if observation.lastOpponentAction == 0:
        return 1
    elif observation.lastOpponentAction == 1:
        return 2
    else:
        return 0

Overwriting oposite_agent.py


8 Опишем поведение агента который сдвиг на 1 по отношению к прошлому действию опонента

In [12]:
%%writefile shift_one_agent.py

def shift_one_agent(observation, configuration):
    if observation.step == 0:
        return 1
    else:
        return (observation.lastOpponentAction + 1) % 3

Overwriting shift_one_agent.py


9 Опишем поведение агента который сдвиг на случайное число по отношению к прошлому действию опонента

In [13]:
%%writefile shift_random_agen    
import random

def shift_random_agent(observation, configuration):
    if observation.step == 0:
        return 0
    else:
        return (observation.lastOpponentAction + random.randrange(0, 100)) % 2

Overwriting shift_random_agen


10 Опишем поведение агента который случайно с добовлением seed шага выбирает любой элемент

In [14]:
%%writefile seed_step_random_agent.py
import random

def seed_step_random_agent(observation, configuration):
    random.seed(observation.step)
    return random.randrange(0, configuration.signs)

Overwriting seed_step_random_agent.py


11 Опишем поведение агента который случайно с добовлением seed времени выбирает любой элемент

In [15]:
%%writefile seed_time_random_agent.py
import random
import time

def seed_time_random_agent(observation, configuration):
    random.seed(time.time())
    result = random.randrange(0, configuration.signs)    
    return result

Overwriting seed_time_random_agent.py


12 Опишем поведение агента который случайно выберает через nampy random

In [16]:
%%writefile nampy_random_agent.py
import numpy as np

def nampy_random_agent(observation, configuration):
    return np.random.randint(0, configuration.signs)

Overwriting nampy_random_agent.py


Создадим турнир где все играют со всеми

In [19]:
agent_list = [
    "scissors_agent.py",
    "paper_agent.py",
    "rock_scissors_agent.py",
    "rock_paper_agent.py",
    "order_rock_scissors_paper_agent.py",
    "random_agent.py",
    "oposite_agent.py",
    "shift_one_agent.py",
    "shift_random_agent.py",
    "seed_step_random_agent.py",
    "seed_time_random_agent.py",
    "nampy_random_agent.py"
]

result_score = {}

for agent in agent_list:
    result_score[agent] = 0

for current_agent in agent_list:
    for agent in agent_list:
        if current_agent == agent:
            continue
        result = evaluate(
                "rps",
                agents = [current_agent, agent],
                configuration = {"episodeSteps": 101, 'tieRewardThreshold': 1},
                debug = False
            )
        if result[0][0] > result[0][1]:
            result_score[current_agent] += 1
        elif result[0][0] < result[0][1]:
            result_score[agent] += 1
sorted(result_score.items(), key=lambda kv: kv[1], reverse = True)

[('nampy_random_agent.py', 14),
 ('rock_scissors_agent.py', 13),
 ('order_rock_scissors_paper_agent.py', 13),
 ('random_agent.py', 13),
 ('oposite_agent.py', 11),
 ('seed_time_random_agent.py', 11),
 ('shift_random_agent.py', 10),
 ('rock_paper_agent.py', 9),
 ('paper_agent.py', 8),
 ('shift_one_agent.py', 8),
 ('seed_step_random_agent.py', 7),
 ('scissors_agent.py', 6)]

In [18]:
result = evaluate(
    "rps", #environment to use - no need to change
    ["nampy_random_agent.py", "scissors_agent.py"], #agents to evaluate
    configuration={"episodeSteps": 101, 'tieRewardThreshold': 1}, #number of episodes,
    debug = True
)
print(f"result {result}")

result [[7.0, -7.0]]
